In [116]:
import xlb
from xlb.compute_backend import ComputeBackend
from xlb.precision_policy import PrecisionPolicy
from xlb.grid import grid_factory
from xlb.operator.stepper import IncompressibleNavierStokesStepper
from xlb.operator.boundary_condition import (
    FullwayBounceBackBC,
    ZouHeBC,
    ExtrapolationOutflowBC,
)
from xlb.operator.macroscopic import Macroscopic
from xlb.utils import save_fields_vtk, save_image
import warp as wp
import numpy as np
import jax.numpy as jnp
import time
import numpy as np

In [ ]:
# class AneurysmSimulation2D:
#     def __init__(self, omega, grid_shape, velocity_set, backend, precision_policy, resolution):
#         # initialize backend
#         xlb.init(
#             velocity_set=velocity_set,
#             default_backend=backend,
#             default_precision_policy=precision_policy,
#         )
        
#         self.omega = omega
#         self.grid_shape = grid_shape
#         self.velocity_set = velocity_set
#         self.backend = backend
#         self.precision_policy = precision_policy
#         self.resolution = resolution  # Store the resolution
#         self.u_max = 0.04  # Maximum velocity for inlet profile

#         # Create grid using factory
#         self.grid = grid_factory(grid_shape, compute_backend=backend)
        
#         # Setup the simulation
#         self._setup()

#     def _setup(self):
#         self.setup_boundary_conditions()
#         self.setup_stepper()

#     def define_boundary_indices(self):
#         box = self.grid.bounding_box_indices()
#         box_no_edge = self.grid.bounding_box_indices(remove_edges=True)
        
#         # Get inlet/outlet from box_no_edge (similar to how lid-driven cavity gets its lid)
#         inlet = box_no_edge["left"]
#         outlet = box_no_edge["right"]
        
#         # Get walls from box and combine like lid-driven cavity does
#         walls = [box["bottom"][i] + box["top"][i] for i in range(self.velocity_set.d)]
#         walls = np.unique(np.array(walls), axis=-1).tolist()
        
#         # Debug prints
#         print("Box format:", type(box["top"]), len(box["top"]))
#         print("Inlet format:", type(inlet), len(inlet))
#         print("Walls format:", type(walls), len(walls))
        
#         return inlet, outlet, walls

#     def setup_boundary_conditions(self):
#         inlet, outlet, walls = self.define_boundary_indices()
#         # # Compute intersections if needed (for debugging)
#         # overlap_inlet = set(inlet).intersection(walls)
#         # overlap_outlet = set(outlet).intersection(walls)
#         # if overlap_inlet:
#         #     print("Overlap between walls and inlet:", overlap_inlet)
#         # if overlap_outlet:
#         #     print("Overlap between walls and outlet:", overlap_outlet)
        
#         # Walls take priority: use full bounce-back
#         bc_walls = FullwayBounceBackBC(indices=walls)
#         # Then define inlet with Zou-He and outlet with extrapolation outflow
#         bc_inlet = ZouHeBC("velocity", prescribed_value=(self.u_max, 0.0), indices=inlet)
#         bc_outlet = ExtrapolationOutflowBC(indices=outlet)

#         self.boundary_conditions = [bc_walls, bc_inlet, bc_outlet]

#     def setup_stepper(self):
#         self.stepper = IncompressibleNavierStokesStepper(
#             omega=self.omega,
#             grid=self.grid,
#             boundary_conditions=self.boundary_conditions,
#             collision_type="BGK",
#         )
#         self.f_0, self.f_1, self.bc_mask, self.missing_mask = self.stepper.prepare_fields()

#     # def bc_profile(self):
#     #     u_max = self.u_max  # u_max = 0.04
#     #     # For 2D: grid_shape = (width, height)
#     #     H_y = float(self.grid_shape[1] - 1)  # Height in y direction

#     #     @wp.func
#     #     def bc_profile_warp(index: wp.vec2i):
#     #         # index: (x, y)
#     #         # Get the y coordinate and convert to the proper precision
#     #         y = self.precision_policy.store_precision.wp_dtype(index[1])
#     #         # Calculate normalized distance from vertical center
#     #         y_center = y - (H_y / 2.0)
#     #         r_squared = (2.0 * y_center / H_y) ** 2.0
#     #         # Parabolic profile: u = u_max * (1 - r²) for the x-component, y-component is zero
#     #         return wp.vec2(u_max * wp.max(0.0, 1.0 - r_squared), 0.0)

#     #     def bc_profile_jax():
#     #         import jax.numpy as jnp
#     #         # Create an array of y coordinates
#     #         y = jnp.arange(self.grid_shape[1])
#     #         # Calculate normalized distance from the center
#     #         y_center = y - (H_y / 2.0)
#     #         r_squared = (2.0 * y_center / H_y) ** 2.0
#     #         # Compute x-velocity from the parabolic profile, set y velocity to zero
#     #         u_x = u_max * jnp.maximum(0.0, 1.0 - r_squared)
#     #         u_y = jnp.zeros_like(u_x)
#     #         # Return a function or field as needed; here we stack so that the first dimension is velocity component
#     #         return jnp.stack([u_x, u_y])
        
#     #     if self.backend == ComputeBackend.JAX:
#     #         return bc_profile_jax
#     #     elif self.backend == ComputeBackend.WARP:
#     #         return bc_profile_warp


#     def run(self, num_steps, post_process_interval=100):
#         for i in range(num_steps):
#             self.f_0, self.f_1 = self.stepper(self.f_0, self.f_1, self.bc_mask, self.missing_mask, i)
#             self.f_0, self.f_1 = self.f_1, self.f_0

#             if i % post_process_interval == 0 or i == num_steps - 1:
#                 self.post_process(i)

#     def post_process(self, i):
#         if not isinstance(self.f_0, jnp.ndarray):
#             f_0 = wp.to_jax(self.f_0)[..., 0]
#         else:
#             f_0 = self.f_0

#         macro = Macroscopic(
#             compute_backend=ComputeBackend.JAX,
#             precision_policy=self.precision_policy,
#             velocity_set=xlb.velocity_set.D2Q9(precision_policy=self.precision_policy, backend=ComputeBackend.JAX),
#         )

#         rho, u = macro(f_0)
#         rho = rho[:, 1:-1, 1:-1]
#         u = u[:, 1:-1, 1:-1]
#         u_magnitude = (u[0] ** 2 + u[1] ** 2) ** 0.5  # Simple magnitude calculation

#         fields = {"rho": rho[0], "u_x": u[0], "u_y": u[1], "u_magnitude": u_magnitude}
#         save_fields_vtk(fields, timestep=i, prefix="aneurysm")
#         save_image(fields["u_magnitude"], timestep=i, prefix="aneurysm")

#     def plot_boundaries(self):
#         import matplotlib.pyplot as plt
#         # Get the boundary indices (lists of tuples)
#         inlet, outlet, walls = self.define_boundary_indices()
        
#         # Unpack x and y coordinates for each boundary
#         inlet_x, inlet_y = zip(*inlet) if inlet else ([], [])
#         outlet_x, outlet_y = zip(*outlet) if outlet else ([], [])
#         walls_x, walls_y = zip(*walls) if walls else ([], [])
        
#         plt.figure(figsize=(6,6))
#         plt.scatter(inlet_x, inlet_y, color='red', label='Inlet', marker='o')
#         plt.scatter(outlet_x, outlet_y, color='blue', label='Outlet', marker='o')
#         plt.scatter(walls_x, walls_y, color='green', label='Walls', marker='x')
#         plt.xlabel('x')
#         plt.ylabel('y')
#         plt.title('Boundary Conditions')
#         plt.legend()
#         plt.gca().invert_yaxis()  # Optional: invert y axis if that matches your grid orientation
#         plt.show()




In [ ]:
# def aneurysm_simulation_setup() -> AneurysmSimulation2D:
#     # Constants 
#     # TODO: move to more appropriate place
#     mm_to_m = 0.001

#     # Physical parameters
#     # Ideally, for the final protoype, we find the smallest possible values for these parameters
#     # based on the actual physical dimensions of the aneurysm model
#     # to reduce the computational cost
#     vessel_length_m = 10 * mm_to_m    # 10 mm long
#     vessel_width_m = 10 * mm_to_m      # 2 mm wide
#     resolution_m = 0.01 * mm_to_m     # Each grid cell = 0.01 mm
    
#     # Calculate grid dimensions
#     grid_x = int(round(vessel_length_m / resolution_m))
#     grid_y = int(round(vessel_width_m / resolution_m))
#     grid_shape = (grid_x, grid_y)

#     # print(grid_shape)
    
#     # Simulation parameters
#     backend = ComputeBackend.WARP
#     precision_policy = PrecisionPolicy.FP32FP32
#     # kinematic_viscosity = 3.3e-6 * units.m**2/units.s  # Blood viscosity
    
#     # Velocity set for 2D
#     velocity_set = xlb.velocity_set.D2Q9(
#         precision_policy=precision_policy,
#         backend=backend
#     )
    
#     # Calculate relaxation parameter (tau) from physical properties
#     kinematic_viscosity = 3.3e-6  # Blood viscosity in m^2/s
#     dx = resolution_m  # Grid spacing in meters
#     dt = 1e-5  # Time step in seconds
#     nu_lbm = kinematic_viscosity * dt / (dx**2)
#     omega = 1.0 / (3 * nu_lbm + 0.5)

#     print((3 * nu_lbm + 0.5))   # Tau value. should be in range 0.5 to 2.0 for stability (source: https://www.researchgate.net/publication/320000000_Lattice_Boltzmann_Methods_for_Complex_Flows_in_Microgeometries)
#     assert 0.5 <= omega <= 2.0, "Omega value out of range. Should be in range 0.5 to 2.0 for stability"
#     print(f"omega = {omega}")

#     velocity_set = xlb.velocity_set.D2Q9(precision_policy=precision_policy, backend=backend)

#     simulation = AneurysmSimulation2D(omega, grid_shape, velocity_set, backend, precision_policy, resolution_m)
#     return simulation

# aneurysm_simulation = aneurysm_simulation_setup()


1.4899999999999998
omega = 0.6711409395973156
Box format: <class 'list'> 2
Inlet format: <class 'list'> 2
Walls format: <class 'list'> 2
registered bc FullwayBounceBackBC_8465041499257 with id 122
registered bc ZouHeBC_8465041429264 with id 123
registered bc ExtrapolationOutflowBC_8465041428511 with id 124
Module xlb.operator.boundary_condition.boundary_condition 8ffe8ce load on device 'cuda:0' took 1560.81 ms  (compiled)


In [155]:
aneurysm_simulation.run(num_steps=10000, post_process_interval=100)

Saved ./aneurysm_0000000.vtk in 0.053270 seconds.
Saved ./aneurysm_0000100.vtk in 0.047740 seconds.
Saved ./aneurysm_0000200.vtk in 0.054896 seconds.
Saved ./aneurysm_0000300.vtk in 0.047102 seconds.
Saved ./aneurysm_0000400.vtk in 0.045555 seconds.
Saved ./aneurysm_0000500.vtk in 0.043956 seconds.
Saved ./aneurysm_0000600.vtk in 0.051030 seconds.
Saved ./aneurysm_0000700.vtk in 0.044286 seconds.
Saved ./aneurysm_0000800.vtk in 0.069232 seconds.
Saved ./aneurysm_0000900.vtk in 0.043607 seconds.
Saved ./aneurysm_0001000.vtk in 0.043030 seconds.
Saved ./aneurysm_0001100.vtk in 0.043198 seconds.
Saved ./aneurysm_0001200.vtk in 0.043704 seconds.
Saved ./aneurysm_0001300.vtk in 0.045508 seconds.
Saved ./aneurysm_0001400.vtk in 0.044191 seconds.
Saved ./aneurysm_0001500.vtk in 0.042801 seconds.
Saved ./aneurysm_0001600.vtk in 0.043759 seconds.
Saved ./aneurysm_0001700.vtk in 0.046002 seconds.
Saved ./aneurysm_0001800.vtk in 0.044642 seconds.
Saved ./aneurysm_0001900.vtk in 0.046382 seconds.


<Figure size 640x480 with 0 Axes>

In [189]:
import xlb
from xlb.compute_backend import ComputeBackend
from xlb.precision_policy import PrecisionPolicy
from xlb.grid import grid_factory
from xlb.operator.stepper import IncompressibleNavierStokesStepper
from xlb.operator.boundary_condition import HalfwayBounceBackBC, EquilibriumBC, ExtrapolationOutflowBC
from xlb.operator.macroscopic import Macroscopic
from xlb.utils import save_fields_vtk, save_image
import xlb.velocity_set
import warp as wp
import jax.numpy as jnp
import numpy as np


class AneurysmSimulation2D:
    def __init__(self, omega, grid_shape, velocity_set, backend, precision_policy, resolution):
        # initialize backend
        xlb.init(
            velocity_set=velocity_set,
            default_backend=backend,
            default_precision_policy=precision_policy,
        )

        self.grid_shape = grid_shape
        self.velocity_set = velocity_set
        self.backend = backend
        self.precision_policy = precision_policy
        self.omega = omega
        self.resolution = resolution
        self.u_max = 0.04  # Maximum inlet velocity
        self.boundary_conditions = []

        # Create grid using factory
        self.grid = grid_factory(grid_shape, compute_backend=backend)
        self._setup()

    def _setup(self):
        self.setup_boundary_conditions()
        self.setup_stepper()
        # Initialize fields using the stepper
        self.f_0, self.f_1, self.bc_mask, self.missing_mask = self.stepper.prepare_fields()

    def define_boundary_indices(self):
        # Get box indices with and without edges
        box = self.grid.bounding_box_indices()
        box_no_edge = self.grid.bounding_box_indices(remove_edges=True)
        
        # Get inlet (left side without corners)
        inlet = box_no_edge["left"]
        
        # Get outlet (right side without corners)
        outlet = box_no_edge["right"]
        
        # Combine top and bottom walls
        walls = [box["top"][i] + box["bottom"][i] for i in range(self.velocity_set.d)]
        walls = np.unique(np.array(walls), axis=-1).tolist()
        
        # Debug prints
        print("Inlet:", inlet)
        print("Outlet:", outlet)
        print("Walls:", walls)
        
        return inlet, outlet, walls

    def setup_boundary_conditions(self):
        inlet, outlet, walls = self.define_boundary_indices()
        
        # Walls: no-slip boundary condition
        bc_walls = FullwayBounceBackBC(indices=walls)
        
        # Inlet: constant velocity profile
        bc_inlet = ZouHeBC("velocity", prescribed_value=(self.u_max, 0.0), indices=inlet)
        
        # Outlet: zero-gradient outflow
        bc_outlet = ExtrapolationOutflowBC(indices=outlet)
        
        self.boundary_conditions = [bc_walls, bc_inlet, bc_outlet]

    def setup_stepper(self):
        self.stepper = IncompressibleNavierStokesStepper(
            omega=self.omega,
            grid=self.grid,
            boundary_conditions=self.boundary_conditions,
            collision_type="KBC",
        )

    def run(self, num_steps, post_process_interval=100):
        for i in range(num_steps):
            self.f_0, self.f_1 = self.stepper(self.f_0, self.f_1, self.bc_mask, self.missing_mask, i)
            self.f_0, self.f_1 = self.f_1, self.f_0

            if i % post_process_interval == 0 or i == num_steps - 1:
                self.post_process(i)

    def post_process(self, i):
        # Write the results. We'll use JAX backend for the post-processing
        if not isinstance(self.f_0, jnp.ndarray):
            # If the backend is warp, we need to drop the last dimension added by warp for 2D simulations
            f_0 = wp.to_jax(self.f_0)[..., 0]
        else:
            f_0 = self.f_0

        macro = Macroscopic(
            compute_backend=ComputeBackend.JAX,
            precision_policy=self.precision_policy,
            velocity_set=xlb.velocity_set.D2Q9(precision_policy=self.precision_policy, backend=ComputeBackend.JAX),
        )

        rho, u = macro(f_0)

        # remove boundary cells
        rho = rho[:, 1:-1, 1:-1]
        u = u[:, 1:-1, 1:-1]
        u_magnitude = (u[0] ** 2 + u[1] ** 2) ** 0.5

        fields = {"rho": rho[0], "u_x": u[0], "u_y": u[1], "u_magnitude": u_magnitude}

        save_fields_vtk(fields, timestep=i, prefix="lid_driven_cavity")
        save_image(fields["u_magnitude"], timestep=i, prefix="lid_driven_cavity")


def aneurysm_simulation_setup() -> AneurysmSimulation2D:
    # Constants 
    # TODO: move to more appropriate place
    mm_to_m = 0.001

    # Physical parameters
    # Ideally, for the final protoype, we find the smallest possible values for these parameters
    # based on the actual physical dimensions of the aneurysm model
    # to reduce the computational cost
    vessel_length_m = 10 * mm_to_m    # 10 mm long
    vessel_width_m = 10 * mm_to_m      # 2 mm wide
    resolution_m = 0.01 * mm_to_m     # Each grid cell = 0.01 mm
    
    # Calculate grid dimensions
    grid_x = int(round(vessel_length_m / resolution_m))
    grid_y = int(round(vessel_width_m / resolution_m))
    grid_shape = (grid_x, grid_y)

    # print(grid_shape)
    
    # Simulation parameters
    backend = ComputeBackend.WARP
    precision_policy = PrecisionPolicy.FP32FP32
    # kinematic_viscosity = 3.3e-6 * units.m**2/units.s  # Blood viscosity
    
    # Velocity set for 2D
    velocity_set = xlb.velocity_set.D2Q9(
        precision_policy=precision_policy,
        backend=backend
    )
    
    # Calculate relaxation parameter (tau) from physical properties
    kinematic_viscosity = 3.3e-6  # Blood viscosity in m^2/s
    dx = resolution_m  # Grid spacing in meters
    dt = 1e-6  # Time step in seconds
    nu_lbm = kinematic_viscosity * dt / (dx**2)
    omega = 1.0 / (3 * nu_lbm + 0.5)

    print((3 * nu_lbm + 0.5))   # Tau value. should be in range 0.5 to 2.0 for stability (source: https://www.researchgate.net/publication/320000000_Lattice_Boltzmann_Methods_for_Complex_Flows_in_Microgeometries)
    assert 0.5 <= omega <= 2.0, "Omega value out of range. Should be in range 0.5 to 2.0 for stability"
    print(f"omega = {omega}")

    velocity_set = xlb.velocity_set.D2Q9(precision_policy=precision_policy, backend=backend)

    simulation = AneurysmSimulation2D(omega, grid_shape, velocity_set, backend, precision_policy, resolution_m)
    return simulation

aneurysm_simulation = aneurysm_simulation_setup()


0.599
omega = 1.669449081803005
Inlet: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [190]:
aneurysm_simulation.run(num_steps=10000, post_process_interval=100)

Module xlb.operator.stepper.nse_stepper f5b6049 load on device 'cuda:0' took 12467.40 ms  (compiled)
Saved ./lid_driven_cavity_0000000.vtk in 0.057734 seconds.
Saved ./lid_driven_cavity_0000100.vtk in 0.053305 seconds.
Saved ./lid_driven_cavity_0000200.vtk in 0.051515 seconds.
Saved ./lid_driven_cavity_0000300.vtk in 0.048459 seconds.
Saved ./lid_driven_cavity_0000400.vtk in 0.047534 seconds.
Saved ./lid_driven_cavity_0000500.vtk in 0.046505 seconds.
Saved ./lid_driven_cavity_0000600.vtk in 0.047342 seconds.
Saved ./lid_driven_cavity_0000700.vtk in 0.046488 seconds.
Saved ./lid_driven_cavity_0000800.vtk in 0.046600 seconds.
Saved ./lid_driven_cavity_0000900.vtk in 0.047144 seconds.
Saved ./lid_driven_cavity_0001000.vtk in 0.045559 seconds.
Saved ./lid_driven_cavity_0001100.vtk in 0.045498 seconds.
Saved ./lid_driven_cavity_0001200.vtk in 0.045728 seconds.
Saved ./lid_driven_cavity_0001300.vtk in 0.046644 seconds.
Saved ./lid_driven_cavity_0001400.vtk in 0.045368 seconds.
Saved ./lid_dr

<Figure size 640x480 with 0 Axes>

In [ ]:
import xlb
from xlb.compute_backend import ComputeBackend
from xlb.precision_policy import PrecisionPolicy
from xlb.grid import grid_factory
from xlb.operator.stepper import IncompressibleNavierStokesStepper
from xlb.operator.boundary_condition import HalfwayBounceBackBC, EquilibriumBC, ExtrapolationOutflowBC
from xlb.operator.macroscopic import Macroscopic
from xlb.utils import save_fields_vtk, save_image
import xlb.velocity_set
import warp as wp
import jax.numpy as jnp
import numpy as np


class AneurysmSimulation2D:
    def __init__(self, omega, grid_shape, velocity_set, backend, precision_policy, resolution):
        # initialize backend
        xlb.init(
            velocity_set=velocity_set,
            default_backend=backend,
            default_precision_policy=precision_policy,
        )

        self.grid_shape = grid_shape
        self.velocity_set = velocity_set
        self.backend = backend
        self.precision_policy = precision_policy
        self.omega = omega
        self.resolution = resolution
        self.u_max = 0.04  # Maximum inlet velocity
        self.boundary_conditions = []

        # Create grid using factory
        self.grid = grid_factory(grid_shape, compute_backend=backend)
        self._setup()

    def _setup(self):
        self.setup_boundary_conditions()
        self.setup_stepper()
        # Initialize fields using the stepper
        self.f_0, self.f_1, self.bc_mask, self.missing_mask = self.stepper.prepare_fields()

    def define_boundary_indices(self):
        # Get box indices with and without edges
        box = self.grid.bounding_box_indices()
        box_no_edge = self.grid.bounding_box_indices(remove_edges=True)

        x, y = self.grid_shape

        mid_horizontal = int(round(y / 2))
        pixel_width = 0.002 * self.resolution

        loc_upper = round(mid_horizontal + pixel_width) 
        loc_lower = round(mid_horizontal - pixel_width)

        print("loc_upper:", loc_upper)
        print("loc_lower:", loc_lower)

        wall_width = [i for i in range(x)]
        array_upper = [loc_upper for i in range(x)]
        array_lower = [loc_lower for i in range(x)] 

        # Combine into 2D arrays in required format
        walls = [wall_width + wall_width,  # x coordinates repeated for upper and lower walls
                array_upper + array_lower]    # y coordinates for lower and upper walls
        

        # Get inlet/outlet from box_no_edge (similar to how lid-driven cavity gets its lid)
        x_array_left = [0 for i in range(loc_lower + 1, loc_upper - 1)]
        x_array_right = [x - 1 for i in range(loc_lower + 1, loc_upper - 1)]
        y_array = [i for i in range(loc_lower + 1, loc_upper - 1)]

        inlet = [x_array_left, y_array]
        outlet = [x_array_right, y_array]



        # # Get inlet (left side without corners)
        # inlet = box_no_edge["left"]
        
        # # Get outlet (right side without corners)
        # outlet = box_no_edge["right"]
        
        # # Combine top and bottom walls
        # walls = [box["top"][i] + box["bottom"][i] for i in range(self.velocity_set.d)]
        # walls = np.unique(np.array(walls), axis=-1).tolist()
        
        # Debug prints
        print("Inlet:", inlet)
        print("Outlet:", outlet)
        print("Walls:", walls)
        
        return inlet, outlet, walls

    def setup_boundary_conditions(self):
        inlet, outlet, walls = self.define_boundary_indices()
        
        # Walls: no-slip boundary condition
        bc_walls = FullwayBounceBackBC(indices=walls)
        
        # Inlet: constant velocity profile
        bc_inlet = ZouHeBC("velocity", prescribed_value=(self.u_max, 0.0), indices=inlet)
        
        # Outlet: zero-gradient outflow
        bc_outlet = ExtrapolationOutflowBC(indices=outlet)
        
        self.boundary_conditions = [bc_walls, bc_inlet, bc_outlet]

    def setup_stepper(self):
        self.stepper = IncompressibleNavierStokesStepper(
            omega=self.omega,
            grid=self.grid,
            boundary_conditions=self.boundary_conditions,
            collision_type="KBC",
        )

    def run(self, num_steps, post_process_interval=100):
        for i in range(num_steps):
            self.f_0, self.f_1 = self.stepper(self.f_0, self.f_1, self.bc_mask, self.missing_mask, i)
            self.f_0, self.f_1 = self.f_1, self.f_0

            if i % post_process_interval == 0 or i == num_steps - 1:
                self.post_process(i)

    def post_process(self, i):
        # Write the results. We'll use JAX backend for the post-processing
        if not isinstance(self.f_0, jnp.ndarray):
            # If the backend is warp, we need to drop the last dimension added by warp for 2D simulations
            f_0 = wp.to_jax(self.f_0)[..., 0]
        else:
            f_0 = self.f_0

        macro = Macroscopic(
            compute_backend=ComputeBackend.JAX,
            precision_policy=self.precision_policy,
            velocity_set=xlb.velocity_set.D2Q9(precision_policy=self.precision_policy, backend=ComputeBackend.JAX),
        )

        rho, u = macro(f_0)

        # remove boundary cells
        rho = rho[:, 1:-1, 1:-1]
        u = u[:, 1:-1, 1:-1]
        u_magnitude = (u[0] ** 2 + u[1] ** 2) ** 0.5

        fields = {"rho": rho[0], "u_x": u[0], "u_y": u[1], "u_magnitude": u_magnitude}

        save_fields_vtk(fields, timestep=i, prefix="lid_driven_cavity")
        save_image(fields["u_magnitude"], timestep=i, prefix="lid_driven_cavity")


def aneurysm_simulation_setup() -> AneurysmSimulation2D:
    # Constants 
    # TODO: move to more appropriate place
    mm_to_m = 0.001

    # Physical parameters
    # Ideally, for the final protoype, we find the smallest possible values for these parameters
    # based on the actual physical dimensions of the aneurysm model
    # to reduce the computational cost
    vessel_length_m = 10 * mm_to_m    # 10 mm long
    vessel_width_m = 10 * mm_to_m      # 10 mm wide
    resolution_m = 0.02 * mm_to_m     # Each grid cell = 0.01 mm
    
    # Calculate grid dimensions
    grid_x = int(round(vessel_length_m / resolution_m))
    grid_y = int(round(vessel_width_m / resolution_m))
    grid_shape = (grid_x, grid_y)

    print(grid_shape)
    
    # Simulation parameters
    backend = ComputeBackend.WARP
    precision_policy = PrecisionPolicy.FP32FP32
    # kinematic_viscosity = 3.3e-6 * units.m**2/units.s  # Blood viscosity
    
    # Velocity set for 2D
    velocity_set = xlb.velocity_set.D2Q9(
        precision_policy=precision_policy,
        backend=backend
    )
    
    # Calculate relaxation parameter (tau) from physical properties
    kinematic_viscosity = 3.3e-6  # Blood viscosity in m^2/s
    dx = resolution_m  # Grid spacing in meters
    dt = 1e-6  # Time step in seconds
    nu_lbm = kinematic_viscosity * dt / (dx**2)
    omega = 1.0 / (3 * nu_lbm + 0.5)

    print((3 * nu_lbm + 0.5))   # Tau value. should be in range 0.5 to 2.0 for stability (source: https://www.researchgate.net/publication/320000000_Lattice_Boltzmann_Methods_for_Complex_Flows_in_Microgeometries)
    assert 0.5 <= omega <= 2.0, "Omega value out of range. Should be in range 0.5 to 2.0 for stability"
    print(f"omega = {omega}")

    velocity_set = xlb.velocity_set.D2Q9(precision_policy=precision_policy, backend=backend)

    simulation = AneurysmSimulation2D(omega, grid_shape, velocity_set, backend, precision_policy, resolution_m)
    return simulation

aneurysm_simulation = aneurysm_simulation_setup()
